In [2]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import pandas_udf
import pandas as pd

In [3]:
spark = SparkSession \
          .builder \
          .master('local[*]') \
          .appName('test') \
          .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 23:03:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Measurements

In [4]:
measurements_schema = types.StructType([
  types.StructField('day', types.DateType(), nullable=False),
  types.StructField('interval', types.IntegerType(), nullable=False),
  types.StructField('detid', types.StringType(), nullable=False),
  types.StructField('flow', types.IntegerType(), nullable=True),
  types.StructField('occ', types.FloatType(), nullable=True),
  types.StructField('error', types.IntegerType(), nullable=True),
  types.StructField('city', types.StringType(), nullable=False),
  types.StructField('speed', types.FloatType(), nullable=True)
])

In [9]:
measurements_df = spark.read \
                    .option("header", True) \
                    .schema(measurements_schema) \
                    .csv('./data/measurements_test.csv')

In [ ]:
# @pandas_udf(types.TimestampType())
# def get_timestamp(days: pd.Series, intervals: pd.Series) -> pd.Series:
#   dts = pd.to_datetime(days)
#   deltas = pd.to_timedelta(intervals, unit='s')
#   return dts + deltas

In [10]:
@pandas_udf(types.IntegerType())
def get_hour(intervals: pd.Series) -> pd.Series:
  # deltas = pd.to_timedelta(intervals, unit='s')
  # return deltas.dt.components.hours
  return intervals.floordiv(3600)

In [ ]:
measurements_df = measurements_df \
                    .withColumn('hour', get_hour('interval')) \
                    .drop('interval')

## Detectors

In [14]:
detectors_schema = types.StructType([
  types.StructField('detid', types.StringType()),
  types.StructField('length', types.DoubleType()),
  types.StructField('pos', types.DoubleType()),
  types.StructField('fclass', types.StringType()),
  types.StructField('road', types.StringType()),
  types.StructField('limit', types.IntegerType()),
  types.StructField('citycode', types.StringType()),
  types.StructField('lanes', types.IntegerType()),
  types.StructField('linkid', types.IntegerType()),
  types.StructField('long', types.DoubleType()),
  types.StructField('lat', types.DoubleType())
])

In [15]:
detectors_df = spark.read \
                .option("header", True) \
                .schema(detectors_schema) \
                .csv('./data/detectors_public.csv')

In [16]:
detectors_df.show()

+------+-----------------+-----------------+-----------+----------------+-----+--------+-----+------+----------+----------+
| detid|           length|              pos|     fclass|            road|limit|citycode|lanes|linkid|      long|       lat|
+------+-----------------+-----------------+-----------+----------------+-----+--------+-----+------+----------+----------+
|U1-52G|0.196036689548578|0.005511921838942|  secondary|Gögginger Straße|   50|augsburg|    1|    72|10.8895527| 48.359957|
|U1-51G|0.130039071784385|0.004013209518671|  secondary|Gögginger Straße|   50|augsburg|    1|    73| 10.889601|48.3599454|
|U1-52L|0.155863214591286|0.022227823915315|  secondary|Gögginger Straße|   50|augsburg|    1|    70|10.8893555|48.3598759|
|U1-51L|0.197675412740605|0.021889369247481|  secondary|Gögginger Straße|   50|augsburg|    1|    71|10.8893958|48.3598617|
| U1-62|0.065182655061287|0.024465133197684|  secondary|   Rosenaustraße|   50|augsburg|    1|    68|10.8893609|48.3605781|
| U1-61|

## Links

In [147]:
spark.stop()